In [1]:
from typing_extensions import Literal

import pandas as pd
import numpy as np
import scipy
from scipy.stats import chi2_contingency
from scipy import stats
from statsmodels.stats.multitest import fdrcorrection
import os
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import graphviz
import copy
from scipy import optimize
from statsmodels.tsa.stattools import adfuller
from pandas.plotting import autocorrelation_plot
from statsmodels.tsa.stattools import adfuller
import datetime
import warnings
warnings.filterwarnings('ignore')


In [2]:
all_reads_agg = pd.DataFrame(columns = ['Aligned_Sequence', 'Reference_Sequence',               'iter',
                'A115_DMSO',               'type',                    0,
                          1,                    2,                    4,
                          8,                   12,                   24,
           'num_of_changes',        'changes_loc'])

path = "D:\\אוניברסיטה\\מעבדה\\BEanalysis\\cindy_res\\observed_allele_freq"
os.chdir(path)
aggregation_functions = {'target':'first','%Reads':'sum'}
for cur_path in os.listdir("."):
    if cur_path in ["Graph.gv", "Graph.gv.pdf"]:
        continue
    print(cur_path)
    a = pd.read_csv(cur_path)
    a.fillna(0, inplace= True)

    rep_a = a[["0h_rep1", "1h_rep1", "2h_rep1", "4h_rep1", "8h_rep1", "12h_rep1", "24h_rep1"]] * 0.01 
    rep_a.columns = [0,1,2,4,8,12,24]
    rep_a["Aligned_Sequence"] = a["target"]
    rep_a["Reference_Sequence"] = a.iloc[0]["target"]
 
    rep_a["iter"] = 1
    rep_a["A115_DMSO"] = cur_path[-8:-4]
    b = cur_path.split("\\")[-1].split("_")[1:3]
    rep_a["type"] = b[0] + "_" + b[1].split(".")[0]
    

    all_reads_agg = all_reads_agg.append(rep_a)

    rep_a = a[["0h_rep2", "1h_rep2", "2h_rep2", "4h_rep2", "8h_rep2", "12h_rep2", "24h_rep2"]] * 0.01 
    rep_a.columns = [0,1,2,4,8,12,24]
    rep_a["Aligned_Sequence"] = a["target"]
    rep_a["Reference_Sequence"] = a.iloc[0]["target"]
    rep_a["iter"] = 2
    rep_a["A115_DMSO"] = cur_path[-8:-4]
    b = cur_path.split("\\")[-1].split("_")[1:3]
    rep_a["type"] = b[0] + "_" + b[1].split(".")[0]
 
    all_reads_agg = all_reads_agg.append(rep_a)


    rep_a = a[["0h_rep3", "1h_rep3", "2h_rep3", "4h_rep3", "8h_rep3", "12h_rep3", "24h_rep3"]] * 0.01 
    rep_a.columns = [0,1,2,4,8,12,24]
    rep_a["Aligned_Sequence"] = a["target"]
    rep_a["Reference_Sequence"] = a.iloc[0]["target"]
    rep_a["iter"] = 3
    rep_a["A115_DMSO"] = cur_path[-8:-4]
    b = cur_path.split("\\")[-1].split("_")[1:3]
    rep_a["type"] = b[0] + "_" + b[1].split(".")[0]

    all_reads_agg = all_reads_agg.append(rep_a)

all_reads_agg = all_reads_agg[~((all_reads_agg[0] == 0) & (all_reads_agg[1] == 0) & (all_reads_agg[2] == 0) & (all_reads_agg[4] == 0) & (all_reads_agg[8] == 0) & (all_reads_agg[12] == 0) & (all_reads_agg[24] == 0))]

20211109_ABE8e_ABE16_A115.csv
20211109_ABE8e_ABE16_DMSO.csv
20211109_ABE8e_ABE9_A115.csv
20211109_ABE8e_ABE9_DMSO.csv
20211109_ABE8e_HEK2_A115.csv
20211109_ABE8e_HEK2_DMSO.csv
20211109_ABE8e_HEK3_A115.csv
20211109_ABE8e_HEK3_DMSO.csv
20211109_ABEmax_ABE16_A115.csv
20211109_ABEmax_ABE16_DMSO.csv
20211109_ABEmax_HEK2_A115.csv
20211109_ABEmax_HEK2_DMSO.csv
20211109_ABEmax_HEK3_A115.csv
20211109_ABEmax_HEK3_DMSO.csv
20211109_BE4max_ABE9_A115.csv
20211109_BE4max_ABE9_DMSO.csv
20211109_BE4max_EMX1_A115.csv
20211109_BE4max_EMX1_DMSO.csv
20211109_BE4max_HEK2_A115.csv
20211109_BE4max_HEK2_DMSO.csv
20211109_BE4max_HEK3_A115.csv
20211109_BE4max_HEK3_DMSO.csv


In [3]:
num_of_changes = []
changes_loc = []
for i in range(all_reads_agg.shape[0]):
    tmp_datapoint = all_reads_agg.iloc[i]
    cur_num_of_changes = 0
    cur_changes_loc = []
    for i in range(len(tmp_datapoint["Aligned_Sequence"])):
        if tmp_datapoint["Aligned_Sequence"][i] != tmp_datapoint["Reference_Sequence"][i]:
            cur_num_of_changes += 1
            cur_changes_loc.append(i)
    num_of_changes.append(cur_num_of_changes)
    changes_loc.append(cur_changes_loc)
all_reads_agg["num_of_changes"] = num_of_changes
all_reads_agg["changes_loc"] = changes_loc


all_reads_agg

,Aligned_Sequence,Reference_Sequence,iter,A115_DMSO,type,0,1,2,4,8,12,24,num_of_changes,changes_loc
0,GGGAATAAATCATAGAATCC,GGGAATAAATCATAGAATCC,1,A115,ABE8e_ABE16,0.931398,0.931352,0.908029,0.911362,0.867019,0.814506,0.745184,0,[]
1,GGGGGTGAATCATAGAATCC,GGGAATAAATCATAGAATCC,1,A115,ABE8e_ABE16,0.007828,0.006585,0.011342,0.019594,0.044229,0.059948,0.085456,3,"[3, 4, 6]"
2,GGGGGTGGATCATAGAATCC,GGGAATAAATCATAGAATCC,1,A115,ABE8e_ABE16,0.0,0.0,0.001148,0.001436,0.009938,0.016062,0.034461,4,"[3, 4, 6, 7]"
3,GGGGGTGAATCGTAGAATCC,GGGAATAAATCATAGAATCC,1,A115,ABE8e_ABE16,0.0,0.0,0.0,0.001825,0.0084,0.01496,0.024545,4,"[3, 4, 6, 11]"
4,GGGGGTGGATCGTAGAATCC,GGGAATAAATCATAGAATCC,1,A115,ABE8e_ABE16,0.0,0.0,0.0,0.0,0.001246,0.004033,0.010034,5,"[3, 4, 6, 7, 11]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2,GGCCTAGACTGAGCACGTGA,GGCCCAGACTGAGCACGTGA,3,DMSO,BE4max_HEK3,0.003992,0.003857,0.004591,0.006307,0.009026,0.014515,0.014012,1,[4]
3,GGCTTAGATTGAGCACGTGA,GGCCCAGACTGAGCACGTGA,3,DMSO,BE4max_HEK3,0.0,0.0,0.0,0.0,0.0,0.001053,0.001249,3,"[3, 4, 8]"
7,GGCCCAGACTGGGCACGTGA,GGCCCAGACTGAGCACGTGA,3,DMSO,BE4max_HEK3,0.003654,0.001612,0.003029,0.001536,0.001982,0.003104,0.003302,1,[11]
9,GGCCCAGACTGAGCACGTGG,GGCCCAGACTGAGCACGTGA,3,DMSO,BE4max_HEK3,0.002745,0.001399,0.001562,0.001096,0.001519,0.001866,0.002142,1,[19]


In [4]:
merged_data = pd.DataFrame(columns = ["Aligned_Sequence", "Reference_Sequence", "#Reads", "%Reads",
                                     "timepoint", "iter", "A115_DMSO", "type"])

target_seqs = pd.read_csv("D:\\אוניברסיטה\\מעבדה\\BEanalysis\\target-site_sequences.csv")
path = "D:\\אוניברסיטה\\מעבדה\\BEanalysis\\BEanalysis-main"
os.chdir(path)
all_timepoints = ["_1hr", "_2hr", "_4hr", "_8hr", "_12hr", "_24hr"]
DMSO_A115 = ["A115", "DMSO"]

counter_tmp = 0
all_edit_types = [name for name in os.listdir(".") if os.path.isdir(name)]
for cur_edit_type in all_edit_types:
    counter_tmp += 1
    print(counter_tmp)
    
    
    os.chdir(cur_edit_type)
    
    ##add hr==0
    for folder in os.listdir("."):
        if "_0hr_" in folder:
            os.chdir(folder)
            for doc in os.listdir("."):
                if "Alleles_frequency_table_around_" in doc:
                    cur_freq_table = pd.read_csv(doc, sep = "\t")
                    
                    all_ref_seqs_trunc = []
                    all_ali_seqs_trunc = []

                    for i in range(cur_freq_table.shape[0]):
                        cur_target_seq = target_seqs[target_seqs["Target site"] == cur_edit_type[cur_edit_type.find("_") + 1:]]["Target sequence"].iloc[0]
                        cur_trunc_loc = cur_freq_table["Reference_Sequence"].iloc[i].find(cur_target_seq)
                        all_ref_seqs_trunc.append(cur_freq_table["Reference_Sequence"].iloc[i][cur_trunc_loc:cur_trunc_loc + len(cur_target_seq)])
                        all_ali_seqs_trunc.append(cur_freq_table["Aligned_Sequence"].iloc[i][cur_trunc_loc:cur_trunc_loc + len(cur_target_seq)])
                    cur_freq_table["Reference_Sequence"] =  all_ref_seqs_trunc
                    cur_freq_table["Aligned_Sequence"] = all_ali_seqs_trunc
                    cur_freq_table = cur_freq_table.groupby(by = ["Aligned_Sequence", "Reference_Sequence", "Unedited", "n_deleted", "n_inserted","n_mutated"]).sum().reset_index()
                    
                    cur_freq_table = cur_freq_table[cur_freq_table["%Reads"] >= 0.1]
                    cur_freq_table["%Reads"] = cur_freq_table["%Reads"] / cur_freq_table["%Reads"].sum()
                    cur_freq_table = cur_freq_table[["Aligned_Sequence", "Reference_Sequence", "#Reads", "%Reads"]]
                    cur_freq_table["timepoint"] = 0
                    cur_freq_table["iter"] = int(folder[-1])
                    cur_freq_table["A115_DMSO"] = "0hr"
                    cur_freq_table["type"] = cur_edit_type
                    merged_data = merged_data.append(cur_freq_table)
            os.chdir("..")
        
    
    for item in DMSO_A115:
        
        all_current_item_exps = [name for name in os.listdir(".") if item in name]
        all_current_item_exps_dict = {1:[], 2:[], 3:[]}
        [all_current_item_exps_dict[int(exp[-1])].append(exp) for exp in all_current_item_exps]

        for cur_rep in all_current_item_exps_dict.keys():

            cur_iter_data = all_current_item_exps_dict[cur_rep]

            for cur_timepoint in all_timepoints:
                for folder in cur_iter_data:
                    
                    if cur_timepoint in folder:
                        os.chdir(folder)

                for doc in os.listdir("."):
                    if "Alleles_frequency_table_around_" in doc:
                        cur_freq_table = pd.read_csv(doc, sep = "\t")
                        
                        all_ref_seqs_trunc = []
                        all_ali_seqs_trunc = []

                        for i in range(cur_freq_table.shape[0]):
                            cur_target_seq = target_seqs[target_seqs["Target site"] == cur_edit_type[cur_edit_type.find("_") + 1:]]["Target sequence"].iloc[0]
                            cur_trunc_loc = cur_freq_table["Reference_Sequence"].iloc[i].find(cur_target_seq)
                            all_ref_seqs_trunc.append(cur_freq_table["Reference_Sequence"].iloc[i][cur_trunc_loc:cur_trunc_loc + len(cur_target_seq)])
                            all_ali_seqs_trunc.append(cur_freq_table["Aligned_Sequence"].iloc[i][cur_trunc_loc:cur_trunc_loc + len(cur_target_seq)])
                        cur_freq_table["Reference_Sequence"] =  all_ref_seqs_trunc
                        cur_freq_table["Aligned_Sequence"] = all_ali_seqs_trunc
                        cur_freq_table = cur_freq_table.groupby(by = ["Aligned_Sequence", "Reference_Sequence", "Unedited", "n_deleted", "n_inserted","n_mutated"]).sum().reset_index()
                        
                        cur_freq_table = cur_freq_table[cur_freq_table["%Reads"] >= 0.1]
                        cur_freq_table["%Reads"] = cur_freq_table["%Reads"] / cur_freq_table["%Reads"].sum()
                        cur_freq_table = cur_freq_table[["Aligned_Sequence", "Reference_Sequence", "#Reads", "%Reads"]]
                        cur_freq_table["timepoint"] = int(cur_timepoint[1:cur_timepoint.index("hr")])
                        cur_freq_table["iter"] = cur_rep
                        cur_freq_table["A115_DMSO"] = item
                        cur_freq_table["type"] = cur_edit_type
                        merged_data = merged_data.append(cur_freq_table)
                
                os.chdir("..")
                
    
    os.chdir("..")
merged_data = merged_data.drop(["Aligned_Sequence", "Reference_Sequence"], axis = 1).groupby(by=["iter", "timepoint", "A115_DMSO", "type"]).sum()
merged_data = merged_data.reset_index()


1
2
3
4
5
6
7
8
9
10
11
12


In [5]:
times_to_test = [1,2,4,8,12,24]
all_res_p_vals = pd.DataFrame(columns = ["type", "iter", "A115_DMSO", "seqA", "seqB", "Both", 1,2,4,8,12,24])
for cur_iter in range(1, 4):
    for cur_type in set(all_reads_agg["type"]):
        for cur_A115_DMSO in (["DMSO", "A115"]):
        
            print(datetime.datetime.now(), cur_iter, cur_type, cur_A115_DMSO)
            tmp_data = all_reads_agg[(all_reads_agg["iter"] == cur_iter) & (all_reads_agg["A115_DMSO"] == cur_A115_DMSO) & (all_reads_agg["type"] == cur_type)]
            if tmp_data.shape[0] > 0:
                tmp_data = tmp_data.drop(["iter", "A115_DMSO", "type"], axis = 1)
                tmp_data_singletons= tmp_data[(tmp_data["num_of_changes"] <= 1) & (tmp_data["num_of_changes"] > 0)]
                



                for i in range(tmp_data_singletons.shape[0] - 1):
                    for j in range(i + 1, tmp_data_singletons.shape[0]):
                        if len(set(tmp_data_singletons.iloc[i]["changes_loc"]) & set(tmp_data_singletons.iloc[j]["changes_loc"])) == 0:
                            times_p_values = {}
                            times_p_values["type"] = cur_type
                            times_p_values["iter"] = cur_iter 
                            times_p_values["A115_DMSO"] = cur_A115_DMSO
                            tmp_first_singleton = tmp_data_singletons.iloc[i]
                            tmp_second_singleton = tmp_data_singletons.iloc[j]

                            times_p_values["seqA"]= tmp_first_singleton["Aligned_Sequence"]
                            times_p_values["seqB"]= tmp_second_singleton["Aligned_Sequence"]
                           
                            multi_change = list(tmp_first_singleton["Reference_Sequence"])
                            for loc in tmp_first_singleton["changes_loc"]:
                                multi_change[loc] = tmp_first_singleton["Aligned_Sequence"][loc]
                            for loc in tmp_second_singleton["changes_loc"]:
                                multi_change[loc] = tmp_second_singleton["Aligned_Sequence"][loc]

                            multi_change = ''.join(multi_change)

                            if tmp_data[tmp_data["Aligned_Sequence"] == multi_change].shape[0] > 0:

                                times_to_test = [1, 2, 4, 8, 12, 24]
                                tmp_first_singleton = tmp_data_singletons.iloc[i]
                                tmp_second_singleton = tmp_data_singletons.iloc[j]

                                

                                multi_change = list(tmp_first_singleton["Reference_Sequence"])
                                for loc in tmp_first_singleton["changes_loc"]:
                                    multi_change[loc] = tmp_first_singleton["Aligned_Sequence"][loc]
                                for loc in tmp_second_singleton["changes_loc"]:
                                    multi_change[loc] = tmp_second_singleton["Aligned_Sequence"][loc]

                                multi_change = ''.join(multi_change)
                                times_p_values["Both"]= multi_change

                                if tmp_data[tmp_data["Aligned_Sequence"] == multi_change].shape[0] > 0:
                                    cur_obs_values = tmp_data[tmp_data["Aligned_Sequence"] == multi_change]

                                for cur_timepoint in times_to_test:

                                    cur_tot_abundance = merged_data[(merged_data["iter"] == cur_iter) & (merged_data["type"] == cur_type) & (merged_data["A115_DMSO"] == cur_A115_DMSO) & (merged_data["timepoint"] == cur_timepoint)]["#Reads"].iloc[0]


                                    tmp_first_singleton_cur_timepoint = tmp_first_singleton[cur_timepoint]


                                    tmp_data_for_singleton = tmp_data[tmp_data["num_of_changes"] >tmp_first_singleton["num_of_changes"]]
                                    for k in range(tmp_data_for_singleton.shape[0]):
                                        to_add = True
                                        for loc_of_change in tmp_first_singleton["changes_loc"]:
                                            if tmp_data_for_singleton.iloc[k]["Aligned_Sequence"][loc_of_change] != tmp_first_singleton["Aligned_Sequence"][loc_of_change]:
                                                to_add = False
                                        if to_add:
                                            tmp_first_singleton_cur_timepoint += tmp_data_for_singleton.iloc[k][cur_timepoint]
                                    tmp_first_singleton_cur_timepoint = int(tmp_first_singleton_cur_timepoint * cur_tot_abundance)


                                    tmp_second_singleton_cur_timepoint = tmp_second_singleton[cur_timepoint]
                                    tmp_data_for_singleton = tmp_data[tmp_data["num_of_changes"] >tmp_second_singleton["num_of_changes"]]
                                    for k in range(tmp_data_for_singleton.shape[0]):

                                        to_add = True
                                        for loc_of_change in tmp_second_singleton["changes_loc"]:
                                            if tmp_data_for_singleton.iloc[k]["Aligned_Sequence"][loc_of_change] != tmp_second_singleton["Aligned_Sequence"][loc_of_change]:
                                                to_add = False
                                        if to_add:
                                            tmp_second_singleton_cur_timepoint += tmp_data_for_singleton.iloc[k][cur_timepoint]
                                    tmp_second_singleton_cur_timepoint = int(tmp_second_singleton_cur_timepoint * cur_tot_abundance)

                                    tmp_cur_obs_values_cur_timepoint = cur_obs_values[cur_timepoint].iloc[0]
                                    tmp_data_for_singleton = tmp_data[tmp_data["num_of_changes"] >cur_obs_values["num_of_changes"].iloc[0]]
                                    for k in range(tmp_data_for_singleton.shape[0]):
                                        to_add = True
                                        for loc_of_change in cur_obs_values["changes_loc"].iloc[0]:
                                            if tmp_data_for_singleton.iloc[k]["Aligned_Sequence"][loc_of_change] != cur_obs_values["Aligned_Sequence"].iloc[0][loc_of_change]:
                                                to_add = False
                                        if to_add:
                                            tmp_cur_obs_values_cur_timepoint += tmp_data_for_singleton.iloc[k][cur_timepoint]

                                    cur_obs_values_cur_timepoint = int(tmp_cur_obs_values_cur_timepoint  * cur_tot_abundance)
                                    obs_array = np.array([[cur_obs_values_cur_timepoint, tmp_first_singleton_cur_timepoint - cur_obs_values_cur_timepoint], 
                                              [tmp_second_singleton_cur_timepoint - cur_obs_values_cur_timepoint, cur_tot_abundance - (tmp_first_singleton_cur_timepoint + tmp_second_singleton_cur_timepoint + cur_obs_values_cur_timepoint)]])
                                    
                                    if tmp_first_singleton_cur_timepoint == 0 or tmp_second_singleton_cur_timepoint == 0:
                                        times_p_values[cur_timepoint] = "one allele is missing"
#                                     elif (tmp_first_singleton_cur_timepoint * tmp_second_singleton_cur_timepoint / cur_tot_abundance < 1) and (tmp_cur_obs_values_cur_timepoint < 1):
#                                         times_p_values[cur_timepoint] = "abundance is too small"
                                        
                                    else:
                                        if np.min(chi2_contingency(obs_array)[3]) >=5:
                                            times_p_values[cur_timepoint] = chi2_contingency(obs_array)[1]
                                        else:
                                            times_p_values[cur_timepoint] = "abundance is too small"
                                all_res_p_vals = all_res_p_vals.append(times_p_values, ignore_index= True)
                                

2022-10-28 00:55:08.645582 1 ABE8e_HEK2 DMSO
2022-10-28 00:55:08.763957 1 ABE8e_HEK2 A115
2022-10-28 00:55:08.936915 1 ABEmax_HEK3 DMSO
2022-10-28 00:55:09.009932 1 ABEmax_HEK3 A115
2022-10-28 00:55:09.071966 1 BE4max_HEK2 DMSO
2022-10-28 00:55:09.129966 1 BE4max_HEK2 A115
2022-10-28 00:55:09.195938 1 ABE8e_ABE16 DMSO
2022-10-28 00:55:09.262970 1 ABE8e_ABE16 A115
2022-10-28 00:55:09.367966 1 BE4max_HEK3 DMSO
2022-10-28 00:55:09.419604 1 BE4max_HEK3 A115
2022-10-28 00:55:09.497585 1 ABEmax_ABE16 DMSO
2022-10-28 00:55:09.582575 1 ABEmax_ABE16 A115
2022-10-28 00:55:09.716588 1 BE4max_EMX1 DMSO
2022-10-28 00:55:09.810548 1 BE4max_EMX1 A115
2022-10-28 00:55:09.901588 1 ABE8e_HEK3 DMSO
2022-10-28 00:55:09.999593 1 ABE8e_HEK3 A115
2022-10-28 00:55:10.133547 1 BE4max_ABE9 DMSO
2022-10-28 00:55:10.193544 1 BE4max_ABE9 A115
2022-10-28 00:55:10.248544 1 ABE8e_ABE9 DMSO
2022-10-28 00:55:10.354583 1 ABE8e_ABE9 A115
2022-10-28 00:55:10.486549 1 ABEmax_HEK2 DMSO
2022-10-28 00:55:10.588546 1 ABEmax_HE

In [6]:
####run this####

for cur_iter in range(1, 4):
    for cur_type in set(all_reads_agg["type"]):
        for cur_A115_DMSO in (["DMSO", "A115"]):
        
            print(datetime.datetime.now(), cur_iter, cur_type, cur_A115_DMSO)

            tmp_data = all_reads_agg[(all_reads_agg["iter"] == cur_iter) & (all_reads_agg["A115_DMSO"] == cur_A115_DMSO) & (all_reads_agg["type"] == cur_type)]
            if tmp_data.shape[0] > 0:
                tmp_data = tmp_data.drop(["iter", "A115_DMSO", "type"], axis = 1)
                tmp_data_singletons= tmp_data[(tmp_data["num_of_changes"] >= 2) & (tmp_data["num_of_changes"] > 0)]
                for i in range(tmp_data_singletons.shape[0]):



                    cur_obs_values_all = tmp_data_singletons.iloc[i]
                    cur_reference = cur_obs_values_all["Reference_Sequence"]
                    cur_all_changes = cur_obs_values_all["changes_loc"]
                    for m in range(len(cur_all_changes) - 1):
                        for n in range(m + 1, len(cur_all_changes)):

                            multi_change = list(cur_obs_values_all["Reference_Sequence"])
                            multi_change[cur_all_changes[m]] = cur_obs_values_all["Aligned_Sequence"][cur_all_changes[m]]
                            multi_change[cur_all_changes[n]] = cur_obs_values_all["Aligned_Sequence"][cur_all_changes[n]]

                            multi_change = ''.join(multi_change)
                            if all_res_p_vals[(all_res_p_vals["iter"] == cur_iter) & (all_res_p_vals["A115_DMSO"] == cur_A115_DMSO) & (all_res_p_vals["type"] == cur_type) & (all_res_p_vals["Both"] == multi_change)].shape[0] == 0:
                                print(multi_change)

                                tmp_first_singleton = list(cur_obs_values_all["Reference_Sequence"])
                                tmp_first_singleton[cur_all_changes[m]] = cur_obs_values_all["Aligned_Sequence"][cur_all_changes[m]]
                                first_single_change_loc = [cur_all_changes[m]]
                                tmp_first_singleton = ''.join(tmp_first_singleton)

                                tmp_second_singleton = list(cur_obs_values_all["Reference_Sequence"])
                                tmp_second_singleton[cur_all_changes[n]] = cur_obs_values_all["Aligned_Sequence"][cur_all_changes[n]]
                                second_single_change_loc = [cur_all_changes[n]]
                                tmp_second_singleton = ''.join(tmp_second_singleton)


                                times_p_values = {}
                                times_p_values["type"] = cur_type
                                times_p_values["iter"] = cur_iter 
                                times_p_values["A115_DMSO"] = cur_A115_DMSO                          
                                times_p_values["seqA"]= tmp_first_singleton
                                times_p_values["seqB"]= tmp_second_singleton
                                times_p_values["Both"]= multi_change

                                for cur_timepoint in times_to_test:
                                    cur_tot_abundance = merged_data[(merged_data["iter"] == cur_iter) & (merged_data["type"] == cur_type) & (merged_data["A115_DMSO"] == cur_A115_DMSO) & (merged_data["timepoint"] == cur_timepoint)]["#Reads"].iloc[0]

                                    tmp_first_singleton_cur_timepoint = 0
                                    tmp_data_for_singleton = tmp_data
                                    for k in range(tmp_data_for_singleton.shape[0]):
                                        to_add = True
                                        for loc_of_change in first_single_change_loc:
                                            if tmp_data_for_singleton.iloc[k]["Aligned_Sequence"][loc_of_change] != tmp_first_singleton[loc_of_change]:
                                                to_add = False
                                        if to_add:
                                            tmp_first_singleton_cur_timepoint += tmp_data_for_singleton.iloc[k][cur_timepoint]
                                    tmp_first_singleton_cur_timepoint = int(tmp_first_singleton_cur_timepoint * cur_tot_abundance)

                                    tmp_second_singleton_cur_timepoint = 0
                                    tmp_data_for_singleton = tmp_data
                                    for k in range(tmp_data_for_singleton.shape[0]):
                                        to_add = True
                                        for loc_of_change in second_single_change_loc:
                                            if tmp_data_for_singleton.iloc[k]["Aligned_Sequence"][loc_of_change] != tmp_second_singleton[loc_of_change]:
                                                to_add = False
                                        if to_add:
                                            tmp_second_singleton_cur_timepoint += tmp_data_for_singleton.iloc[k][cur_timepoint]
                                    tmp_second_singleton_cur_timepoint = int(tmp_second_singleton_cur_timepoint * cur_tot_abundance)

                                    tmp_cur_obs_values_cur_timepoint = 0
                                    tmp_data_for_singleton = tmp_data
                                    for k in range(tmp_data_for_singleton.shape[0]):
                                        to_add = True
                                        for loc_of_change in [first_single_change_loc[0], second_single_change_loc[0]]:
                                            if tmp_data_for_singleton.iloc[k]["Aligned_Sequence"][loc_of_change] != multi_change[loc_of_change]:
                                                to_add = False
                                        if to_add:
                                            tmp_cur_obs_values_cur_timepoint += tmp_data_for_singleton.iloc[k][cur_timepoint]

                                    tmp_cur_obs_values_cur_timepoint = int(tmp_cur_obs_values_cur_timepoint * cur_tot_abundance)


                                    obs_array = np.array([[tmp_cur_obs_values_cur_timepoint, tmp_first_singleton_cur_timepoint - tmp_cur_obs_values_cur_timepoint], 
                                              [tmp_second_singleton_cur_timepoint - tmp_cur_obs_values_cur_timepoint, cur_tot_abundance - (tmp_first_singleton_cur_timepoint + tmp_second_singleton_cur_timepoint + tmp_cur_obs_values_cur_timepoint)]])

                                    if tmp_first_singleton_cur_timepoint == 0 or tmp_second_singleton_cur_timepoint == 0:
                                        times_p_values[cur_timepoint] = "one allele is missing"
#                                     elif (tmp_first_singleton_cur_timepoint * tmp_second_singleton_cur_timepoint / cur_tot_abundance < 1) and (tmp_cur_obs_values_cur_timepoint < 1):
#                                         times_p_values[cur_timepoint] = "abundance is too small"
                                    
                                    else:    
                                        if np.min(chi2_contingency(obs_array)[3]) >=5:
                                            times_p_values[cur_timepoint] = chi2_contingency(obs_array)[1]
                                        else:
                                            times_p_values[cur_timepoint] = "abundance is too small"

                                all_res_p_vals = all_res_p_vals.append(times_p_values, ignore_index= True)




2022-10-28 00:55:14.694545 1 ABE8e_HEK2 DMSO
GAGCGCAAAGCATAGACTGC
GAGCACGAAGCATAGACTGC
GAACGCAGAGCATAGACTGC
GAACACGGAGCATAGACTGC
GAACGCAAAGCGTAGACTGC
GAACACGAAGCGTAGACTGC
2022-10-28 00:55:15.188544 1 ABE8e_HEK2 A115
GAGCGCAAAGCATAGACTGC
GAGCACGAAGCATAGACTGC
GAACGCAGAGCATAGACTGC
GAACACGGAGCATAGACTGC
GAACGCAAAGCGTAGACTGC
GAACACGAAGCGTAGACTGC
GAGCACAGAGCATAGACTGC
GAACGCAAGGCATAGACTGC
GAACACGAGGCATAGACTGC
GAACACAGGGCATAGACTGC
GAGCACAAAGCGTAGACTGC
GAACGCAAAGCATGGACTGC
GAACACGAAGCATGGACTGC
GAACACAGAGCGTAGACTGC
GAGCACAAGGCATAGACTGC
2022-10-28 00:55:16.825592 1 ABEmax_HEK3 DMSO
2022-10-28 00:55:16.828612 1 ABEmax_HEK3 A115
2022-10-28 00:55:16.830593 1 BE4max_HEK2 DMSO
GAATATAAAGCATAGACTGC
2022-10-28 00:55:16.889548 1 BE4max_HEK2 A115
GAATATAAAGCATAGACTGC
GAATAGAAAGCATAGACTGC
2022-10-28 00:55:17.053543 1 ABE8e_ABE16 DMSO
GGGGGTAAATCATAGAATCC
GGGGATGAATCATAGAATCC
GGGGATAGATCATAGAATCC
GGGAGTAGATCATAGAATCC
GGGAATGGATCATAGAATCC
GGGGATAAATCGTAGAATCC
GGGAGTAAATCGTAGAATCC
GGGAATGAATCGTAGAATCC
2022-10-

GAAGGCCAAGGGTAGACTGC
GAAGACCGAGGGTAGACTGC
GAAGGCCAAGGATGGACTGC
GAAGACCGAGGATGGACTGC
2022-10-28 00:55:35.011150 3 ABEmax_HEK2 DMSO
2022-10-28 00:55:35.014150 3 ABEmax_HEK2 A115


In [7]:
all_res_p_vals.groupby(["type", "iter", "A115_DMSO", "seqA", "seqB", "Both"]).max().to_csv("D:\\chi.csv")
# all_res_p_vals.sort_values
all_res_p_vals.sort_values(["type", "A115_DMSO", "seqA", "seqB", "Both"]).to_csv("D:\\chi_full.csv")
